In [1]:
# Question 1

In [2]:
import tensorflow as tf

tf.get_logger().setLevel('ERROR') 

# load keras model
model = tf.keras.models.load_model("model_2024_hairstyle.keras")

# convert keras model to tf-lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save tf-lite model
with open("model_2024_hairstyle.tflite", "wb") as f:
    f.write(tflite_model)

# print size of converted model
model_size = len(tflite_model) / (1024 * 1024) 
print(f"Size of the TF-Lite model: {model_size:.2f} MB")


Saved artifact at '/var/folders/_4/617xrc517f5dk8c2t0gbnksr0000gn/T/tmphxrvyec1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5203616768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5203621696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5203866752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5203868688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5204041168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5204038000: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733757540.656042 2405574 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1733757540.663816 2405574 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-09 16:19:00.670060: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/_4/617xrc517f5dk8c2t0gbnksr0000gn/T/tmphxrvyec1
2024-12-09 16:19:00.670344: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-09 16:19:00.670348: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/_4/617xrc517f5dk8c2t0gbnksr0000gn/T/tmphxrvyec1
I0000 00:00:1733757540.682006 2405574 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-12-09 16:19:00.683804: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-09 16:19:01.006610: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder

Size of the TF-Lite model: 76.58 MB


In [3]:
# Question 2

In [4]:
# load tf model
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input Index: {input_details[0]['index']}")
print(f"Output Index: {output_details[0]['index']}")


Input Index: 0
Output Index: 13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# Question 3

In [6]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np

# functions to download and resize image
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# download image
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
image = download_image(url)

# prepare image with target size from prev homework
target_size = (200, 200)  
image = prepare_image(image, target_size)
print("Image prepared successfully!")
print(f"The target size should be: {target_size}")


Image prepared successfully!
The target size should be: (200, 200)


In [7]:
# convert image to numpy array
image_array = np.array(image) 

# normalize array (scale values to [0, 1])
image_array = image_array / 255.0
image_array = image_array.astype(np.float32)

# expand dimensions to add batch size dimension
input_array = np.expand_dims(image_array, axis=0)

print(f"First pixel value, R channel: {input_array[0, 0, 0, 0]:.2f}")


First pixel value, R channel: 0.24


In [8]:
# Question 4

In [9]:
# Add batch dimension
input_data = np.expand_dims(image_array, axis=0)

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(f"Model output: {output_data}")


Model output: [[0.8937725]]


In [10]:
# Question 5